In [1]:
import os
import pandas as pd

### Como linkar os dados das diferentes tabelas?

<img src="../equiColunas.png" alt="Alternative text" />

In [2]:
# Não usar a base de Registros de obitos do IML pq não necessariamente carcateriza moetes violentas/crimes
# Base de feminicídios é um subset da base de homicídios (Feminicídio é o assassinato de uma mulher pelo simples fato de ser mulher)
# enriquecer a base de homicídios com a de feminicídios
# latrocionio não é excludente, pode consolidar junto
# LCSM tbb


In [3]:
dfHD = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_Homicidio_2017a2022.csv",sep="|")
dfMS = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_MorteSuspeita_2013a2022.csv",sep="|")
dfMDIP = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_MDIP_2013a2022.csv",sep="|")
dfLatro = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_Latrocinio_2018a2022.csv",sep="|")
dfLCSM = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_LCSM_2016a2022.csv",sep="|")
dfFem = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_Feminicidio_2015a2022.csv",sep="|")


/var/folders/7t/z5pm4kzs2633wr81yfqjlcj80000gn/T/ipykernel_47772/2876896918.py:1: DtypeWarning: Columns (11,21,26) have mixed types. Specify dtype option on import or set low_memory=False.
  dfHD = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_Homicidio_2017a2022.csv",sep="|")


In [4]:
dfHD = dfHD[(dfHD['DEPARTAMENTO_CIRC']=='DECAP')]
dfHD['uniqueKey'] = dfHD['DP - CIRCUNSCRICAO '].astype(str)+dfHD['NUM_BO'].astype(str)+dfHD['ANO_BO'].astype(str)
dfHD['crime'] = 'Homicídio Doloso'

dfMS = dfMS[(dfMS['DEP_CIRCUNSCRICAO']=='DECAP')]
dfMS['uniqueKey'] = dfMS['DP_CIRCUNSCRICAO'].astype(str)+dfMS['NUM_BO'].astype(str)+dfMS['ANO_BO'].astype(str)
dfMS['crime'] = 'Morte Suspeita'

dfMDIP = dfMDIP[(dfMDIP['DEPARTAMENTO_\nCIRC']=='DECAP')]
dfMDIP['uniqueKey'] = dfMDIP['CIRCUNSCRIÇÃO'].astype(str)+dfMDIP['NUM_BO'].astype(str)+dfMDIP['ano'].astype(str)
dfMDIP['crime'] = 'Morte Decorrente de Intervenção Policial'

dfLatro = dfLatro[(dfLatro['DEPARTAMENTO_CIRC']=='DECAP')]
dfLatro['uniqueKey'] = dfLatro['DP - CIRCUNSCRICAO '].astype(str)+dfLatro['NUM_BO'].astype(str)+dfLatro['ANO_BO'].astype(str)
dfLatro['crime'] = 'Latrocínio'

dfLCSM = dfLCSM[(dfLCSM['DEPARTAMENTO_CIRCUNSCRICAO']=='DECAP')]
dfLCSM['uniqueKey'] = dfLCSM['DP_CIRCUNSCRICAO'].astype(str)+dfLCSM['NUM_BO'].astype(str)+dfLCSM['ANO_BO'].astype(str)
dfLCSM['crime'] = 'Lesão Corporal Seguida de Morte'

dfFem = dfFem[(dfFem['DEPARTAMENTO_CIRCUNSCRICAO']=='DECAP')]
dfFem['uniqueKey'] = dfFem['DP_CIRCUNSCRICAO '].astype(str)+dfFem['NUM_BO'].astype(str)+dfFem['ANO_BO'].astype(str)
dfFem['crime'] = 'Feminicidio'

In [6]:
uniqueHD = [i for i in list(dfHD['uniqueKey']) if i not in list(dfFem['uniqueKey'])]
dfHDUnique = dfHD[dfHD['uniqueKey'].isin(uniqueHD)]
        
len(dfHDUnique)

###### Results #########

# dfMS: 0 overlap
# dfMDIP 0 overlap
# dfLatro 0 overlap
# dfLCSM: 1 overlap
# dfFem: 138 overlaps


3653

In [7]:
print(len(dfHDUnique)+len(dfMS)+len(dfMDIP)+len(dfLatro)+len(dfLCSM)+len(dfFem))

7416


## Renomeando colunas


In [8]:
dfColunas= pd.read_excel(os.getcwd()+f"/../data/colunasFinal.xlsx")

In [9]:
dictFem = dfColunas[['Feminicídio','FinalTable']].dropna().set_index('Feminicídio').to_dict()['FinalTable']
dictHD = dfColunas[['Homicidio','FinalTable']].dropna().set_index('Homicidio').to_dict()['FinalTable']
dictLatro = dfColunas[['Latrocinio','FinalTable']].dropna().set_index('Latrocinio').to_dict()['FinalTable']
dictLCSM = dfColunas[['Lesão Corporal seguida de morte','FinalTable']].dropna().set_index('Lesão Corporal seguida de morte').to_dict()['FinalTable']
dictMS = dfColunas[['Morte Suspeita','FinalTable']].dropna().set_index('Morte Suspeita').to_dict()['FinalTable']
dictMDIP = dfColunas[['MDIP','FinalTable']].dropna().set_index('MDIP').to_dict()['FinalTable']

In [10]:
dfHDRenamed = dfHDUnique.rename(columns=dictHD,inplace=False)[list(dictHD.values())]
dfMSRenamed = dfMS.rename(columns=dictMS,inplace=False)[list(dictMS.values())]
dfMDIPRenamed = dfMDIP.rename(columns=dictMDIP,inplace=False)[list(dictMDIP.values())]
dfLatroRenamed = dfLatro.rename(columns=dictLatro,inplace=False)[list(dictLatro.values())]
dfLCSMRenamed = dfLCSM.rename(columns=dictLCSM,inplace=False)[list(dictLCSM.values())]
dfFemRenamed = dfFem.rename(columns=dictFem,inplace=False)[list(dictFem.values())]

In [11]:
dfFinalMortes= pd.concat([dfHDRenamed,dfMSRenamed,dfMDIPRenamed,dfLatroRenamed,dfLCSMRenamed,dfFemRenamed])
dfFinalMortes.to_csv(os.getcwd()+f"/../data/compiledData/dfMortesCompilado_SemFiltroData.csv",index=False)


## Dados Furto e Roubo

In [255]:
dfFurtoVeic = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_FurtoVeículos_2019a2022.csv",sep="|")
dfRouboVeic = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_RouboVeículos_2019a2022.csv",sep="|")
dfFurtoCel = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_FurtoCelular_2019a2022.csv",sep="|")
dfRouboCel = pd.read_csv(os.getcwd()+f"/../data/compiledData/fullDf_RouboCelular_2019a2022.csv",sep="|")

In [256]:
dfFurtoVeic["ARQUIVO"] = 'Furto Veículo'
dfRouboVeic["ARQUIVO"] = 'Roubo Veículo'
dfFurtoCel["ARQUIVO"] = 'Furto Celular'
dfRouboCel["ARQUIVO"] = 'Roubo Celular'

In [257]:
dfFinalRoubosFurtos = pd.concat([dfFurtoVeic,dfRouboVeic,dfFurtoCel,dfRouboCel])
dfFinalRoubosFurtos.to_csv(os.getcwd()+f"/../data/compiledData/dfRoubosFurtosCompilado.csv",index=False)

In [258]:
len(dfFinalRoubosFurtos)

1097855

In [259]:
dfFinalRoubosFurtos.ARQUIVO.value_counts()

Roubo Celular    438537
Furto Celular    360722
Furto Veículo    156600
Roubo Veículo    141996
Name: ARQUIVO, dtype: int64